In [1]:
import sys
import os
sys.path.append('../') # root directory

In [2]:
from constants import constant
import pandas as pd
import numpy as np
import constants.constant as constant
import utils.commonutils as utils
import utils.scraperutils as scraperutils
from constants.namedtuples import Column
from write.JsonFileWriter import write
from read.JsonFileReader import parse
from pathlib import Path
from tqdm import tqdm
from itertools import chain

In [3]:
os.listdir('data')

['MorningStarStat.json',
 'ARK_INNOVATION_ETF_ARKK_HOLDINGS.csv',
 '.ipynb_checkpoints',
 'ARKK.json']

In [4]:
stocks = parse('data/ARKK.json')
KEY_STATS = 'keyStats'
FIN_STATS = 'financeStats'
TICKER = 'symbol'
PERIOD_ALL = 100
NULL = '—'
len(stocks)

36

In [5]:
KEY_STAT_NAME = 'name'
KEY_STAT_VALUES = 'values'
KEY_STAT_PERIOD = 'period'
KEY_STAT_VALUE = 'value'

In [28]:
stat_name = 'Operating Margin'
stock_ticker = 'NTLA'

## Retrieve Stock, Stat & Values from Raw Data

In [29]:
def get_stock(stocks, ticker):
    '''
    return stock based on ticker
    '''
    for stock in stocks:
        if stock[TICKER] == ticker:
            return stock

In [30]:
def get_stat(stats, name):
    '''
    return stat based on name & stats passed
    '''
    for stat in stats:
        if stat[KEY_STAT_NAME] == name:
            return stat

In [31]:
def get_value(stat, period=-1):
    '''
    return stat values for the passed stat, using period as the index. -1 returns TTM. 100 returns all. 
    '''
    if period == PERIOD_ALL:
        return stat[KEY_STAT_VALUES]
    else:
        return stat[KEY_STAT_VALUES][period]

In [32]:
def get_stat_value(stock, stat_type, stat_name, period=-1):
    stat = get_stat(stock[stat_type], stat_name)
    value = get_value(stat, period=period)
    return value

In [33]:
stock

In [34]:
stock = get_stock(stocks, stock_ticker)
stat = get_stat(stock[KEY_STATS], stat_name)
values = get_value(stat, period=PERIOD_ALL)
values

[{'period': '2011-12', 'value': '—'},
 {'period': '2012-12', 'value': '—'},
 {'period': '2013-12', 'value': '—'},
 {'period': '2014-12', 'value': '—'},
 {'period': '2015-12', 'value': '-221.86'},
 {'period': '2016-12', 'value': '-195.15'},
 {'period': '2017-12', 'value': '-266.32'},
 {'period': '2018-12', 'value': '-298.58'},
 {'period': '2019-12', 'value': '-246.78'},
 {'period': '2020-12', 'value': '-235.51'},
 {'period': 'TTM', 'value': '-857.78'}]

In [ ]:
get_stat_value(stat)

In [12]:
pd.Series([v[KEY_STAT_VALUE] for v in values], dtype='float64')

NameError: name 'values' is not defined

## ARKK Analysis

In [29]:
arkk = pd.read_csv('data/ARK_INNOVATION_ETF_ARKK_HOLDINGS.csv')

In [30]:
# sort the arkk df based on stocks loaded from json
arkk.index = arkk['ticker']
tickers = pd.Series([s[TICKER] for s in stocks])
print(tickers.shape, arkk.shape)
arkk = arkk.loc[tickers]
arkk.shape

(36,) (39, 8)


(36, 8)

In [112]:
# Efficiency
ROIC = 'Return on Invested Capital %'
OPERATING_MARGIN = 'Operating Margin'
GROSS_MARGIN = 'Gross Margin'
NET_MARGIN = 'Net Margin %'

# Growth
OPERATING_CF_GROWTH = 'Operating Cash Flow Growth % YOY'
REVENUE_GROWTH_1 = 'Revenue % Year over Year'
REVENUE_GROWTH_3 = 'Revenue % 3-Year Average'
REVENUE_GROWTH_5 = 'Revenue % 5-Year Average'
OPERATING_INCOME_GROWTH_1 = 'Operating Income % Year over Year'
OPERATING_INCOME_GROWTH_3 = 'Operating Income % 3-Year Average'
OPERATING_INCOME_GROWTH_5 = 'Operating Income % 5-Year Average'

# Financial Risks
DEBT_EQUITY = 'Debt/Equity'

OPERATING_CF = 'Operating Cash Flow'
OPERATING_INC = 'Operating Income'

In [47]:
arkk['roic'] = pd.Series(
    [get_stat_value(s, KEY_STATS, ROIC)['value'] for s in stocks]
).replace({NULL: float('nan')}).astype('float64').values

arkk['op_margin'] = pd.Series(
    [get_stat_value(s, KEY_STATS, OPERATING_MARGIN)['value'] for s in stocks]
).replace({NULL: float('nan')}).astype('float64').values

arkk['gross_margin'] = pd.Series(
    [get_stat_value(s, KEY_STATS, GROSS_MARGIN)['value'] for s in stocks]
).replace({NULL: float('nan')}).astype('float64').values

arkk['net_margin'] = pd.Series(
    [get_stat_value(s, KEY_STATS, NET_MARGIN)['value'] for s in stocks]
).replace({NULL: float('nan')}).astype('float64').values

In [60]:
arkk['op_inc_1_yr_growth'] = pd.Series(
    [get_stat_value(s, KEY_STATS, OPERATING_INCOME_GROWTH_1, period=-2)['value'] for s in stocks]
).replace({NULL: float('nan')}).astype('float64').values

arkk['rev_3_yr_growth'] = pd.Series(
    [get_stat_value(s, KEY_STATS, REVENUE_GROWTH_3, period=-2)['value'] for s in stocks]
).replace({NULL: float('nan')}).astype('float64').values

arkk['rev_1_yr_growth'] = pd.Series(
    [get_stat_value(s, KEY_STATS, REVENUE_GROWTH_1, period=-2)['value'] for s in stocks]
).replace({NULL: float('nan')}).astype('float64').values

arkk['ocf_yr_growth'] = pd.Series(
    [get_stat_value(s, KEY_STATS, OPERATING_CF_GROWTH, period=-2)['value'] for s in stocks]
).replace({NULL: float('nan')}).astype('float64').values

In [113]:
arkk['ocf'] = pd.Series(
    [get_stat_value(s, FIN_STATS, OPERATING_CF, period=-1)['value'] for s in stocks]
).replace({NULL: float('nan')}).values#.replace({',':''}).astype('float64').values
arkk['op_income'] = pd.Series(
    [get_stat_value(s, FIN_STATS, OPERATING_INC, period=-1)['value'] for s in stocks]
).replace({NULL: float('nan')}).values#.replace({',':''}).astype('float64').values

In [119]:
arkk.to_csv('export/arkk/arkk_analysis_20220305.csv', index=False)

In [62]:
arkk.isna().sum(axis=0)

date                   0
fund                   0
company                0
ticker                 0
cusip                  0
shares                 0
market value ($)       0
weight (%)             0
roic                   0
op_margin              1
gross_margin           5
net_margin             2
op_inc_3_yr_growth    35
rev_3_yr_growth       11
ocf_yr_growth         23
op_inc_1_yr_growth    31
rev_1_yr_growth        2
dtype: int64

In [85]:
arkk_stat_names = ['rev_1_yr_growth', 'rev_3_yr_growth', 'ocf_yr_growth', 'op_inc_1_yr_growth', 'gross_margin', 'op_margin', 'net_margin', 'roic']
arkk_stats = pd.DataFrame({col: arkk[col].describe() for col in arkk_stat_names})

In [97]:
arkk_stats

,rev_1_yr_growth,rev_3_yr_growth,ocf_yr_growth,op_inc_1_yr_growth,gross_margin,op_margin,net_margin,roic
count,34.000000,25.000000,13.000000,5.000000,31.000000,35.000000,34.000000,36.000000
mean,68.736765,72.206000,183.563077,1119.942000,58.711290,-54.608857,-74.767353,-12.756944
std,74.100075,109.434002,291.273947,2225.192632,19.785143,214.531771,170.678476,27.197398
min,-18.120000,-7.980000,-18.060000,10.650000,24.140000,-857.780000,-854.100000,-98.590000
25%,24.112500,30.460000,14.870000,68.000000,43.710000,-60.280000,-69.142500,-26.410000
50%,51.225000,54.990000,48.900000,197.990000,60.720000,-32.080000,-31.640000,-9.125000
75%,93.715000,73.300000,132.450000,225.780000,74.760000,4.650000,2.320000,3.815000
max,325.810000,564.100000,868.570000,5097.290000,90.750000,714.500000,63.200000,33.770000


In [120]:
arkk.shape

(36, 19)

In [121]:
arkk[arkk['rev_1_yr_growth'].isna()]

,date,fund,company,ticker,cusip,shares,market value ($),weight (%),roic,op_margin,gross_margin,net_margin,op_inc_3_yr_growth,rev_3_yr_growth,ocf_yr_growth,op_inc_1_yr_growth,rev_1_yr_growth,ocf,op_income
ticker,,,,,,,,,,,,,,,,,,,
CGEN,02/22/2022,ARKK,COMPUGEN LTD,CGEN,M25722105,"5,651,408","$16,276,055.04",0.15%,-30.47,-442.31,90.75,-426.51,NaN,NaN,NaN,NaN,NaN,-30,-35
CRSP,02/22/2022,ARKK,CRISPR THERAPEUTICS AG,CRSP,H17182108,"6,095,582","$355,311,474.78",3.20%,17.36,40.82,NaN,41.28,NaN,564.1,NaN,NaN,NaN,539,374


In [98]:
arkk.to_csv('export/arkk_analysis_20220301.csv', index=False)
arkk_stats.to_csv('export/arkk_stats_20220301.csv', index=False)

In [99]:
arkk[arkk['rev_3_yr_growth'] < 0]

,date,fund,company,ticker,cusip,shares,market value ($),weight (%),roic,op_margin,gross_margin,net_margin,op_inc_3_yr_growth,rev_3_yr_growth,ocf_yr_growth,op_inc_1_yr_growth,rev_1_yr_growth
ticker,,,,,,,,,,,,,,,,,
PACB,02/22/2022,ARKK,PACIFIC BIOSCIENCES OF CALIF,PACB,69404D108,"11,863,154","$136,070,376.38",1.22%,-2.84,-114.29,43.98,-30.39,NaN,-5.49,NaN,NaN,-13.20
SSYS,02/22/2022,ARKK,STRATASYS LTD,SSYS,M85548101,"4,591,664","$118,694,514.40",1.07%,-5.42,-11.25,43.44,-7.92,NaN,-7.98,NaN,NaN,-18.12


In [100]:
arkk

,date,fund,company,ticker,cusip,shares,market value ($),weight (%),roic,op_margin,gross_margin,net_margin,op_inc_3_yr_growth,rev_3_yr_growth,ocf_yr_growth,op_inc_1_yr_growth,rev_1_yr_growth
ticker,,,,,,,,,,,,,,,,,
U,02/22/2022,ARKK,UNITY SOFTWARE INC,U,91332U101,"5,542,691","$549,225,251.19",4.94%,-21.09,-46.08,77.10,-44.77,NaN,NaN,NaN,NaN,42.58
BEAM,02/22/2022,ARKK,BEAM THERAPEUTICS INC,BEAM,07373V105,"6,026,795","$403,915,800.90",3.63%,-65.07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.33
TWOU,02/22/2022,ARKK,2U INC,TWOU,90214J101,"5,745,188","$55,383,612.32",0.50%,-8.27,-16.11,71.19,-18.00,NaN,39.27,NaN,NaN,34.78
MTLS,02/22/2022,ARKK,MATERIALISE NV-ADR,MTLS,57667T100,"3,751,513","$74,430,017.92",0.67%,2.32,3.47,56.90,3.25,NaN,6.13,5.55,NaN,-13.34
TWST,02/22/2022,ARKK,TWIST BIOSCIENCE CORP,TWST,90184D100,"2,178,861","$115,937,193.81",1.04%,-24.06,-122.60,38.77,-112.70,NaN,73.30,NaN,NaN,46.87
FATE,02/22/2022,ARKK,FATE THERAPEUTICS INC,FATE,31189P102,"5,913,842","$195,984,723.88",1.76%,-29.35,-331.08,NaN,-359.67,NaN,97.09,NaN,NaN,194.33
CERS,02/22/2022,ARKK,CERUS CORP,CERS,157085101,"9,861,026","$52,559,268.58",0.47%,-33.59,-39.38,60.72,-41.70,NaN,30.57,NaN,NaN,21.83
CGEN,02/22/2022,ARKK,COMPUGEN LTD,CGEN,M25722105,"5,651,408","$16,276,055.04",0.15%,-30.47,-442.31,90.75,-426.51,NaN,NaN,NaN,NaN,NaN
TDOC,02/22/2022,ARKK,TELADOC HEALTH INC,TDOC,87918A105,"11,023,811","$716,547,715.00",6.45%,-7.34,-32.39,67.87,-43.60,NaN,67.38,NaN,NaN,97.71
